task 1

In [14]:
%pip install beautifulsoup4
%pip install lxml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:

from bs4 import BeautifulSoup
with open("weather.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")
table_rows = soup.find("tbody").find_all("tr")
weather_data = []

for row in table_rows:
    cols = row.find_all("td")
    day = cols[0].text.strip()
    temperature = int(cols[1].text.strip().replace("°C", ""))  # Convert to integer
    condition = cols[2].text.strip()
    weather_data.append((day, temperature, condition))
print("5-Day Weather Forecast:")
for day, temp, cond in weather_data:
    print(f"{day}: {temp}°C, {cond}")
max_temp_day = max(weather_data, key=lambda x: x[1])
print(f"\nHottest day: {max_temp_day[0]} with {max_temp_day[1]}°C")
sunny_days = [day for day, temp, cond in weather_data if cond == "Sunny"]
print(f"Sunny days: {', '.join(sunny_days)}")
average_temp = sum(temp for _, temp, _ in weather_data) / len(weather_data)
print(f"Average Temperature: {average_temp:.2f}°C")


5-Day Weather Forecast:
Monday: 25°C, Sunny
Tuesday: 22°C, Cloudy
Wednesday: 18°C, Rainy
Thursday: 20°C, Partly Cloudy
Friday: 30°C, Sunny

Hottest day: Friday with 30°C
Sunny days: Monday, Friday
Average Temperature: 23.00°C


In [3]:
import requests
import sqlite3
import csv
from bs4 import BeautifulSoup

URL = "https://realpython.github.io/fake-jobs"
DB_NAME = "jobs.db"
TABLE_NAME = "jobs"
def create_table():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_title TEXT,
            company_name TEXT,
            location TEXT,
            description TEXT,
            application_link TEXT,
            UNIQUE(job_title, company_name, location)
        )
    """)
    conn.commit()
    conn.close()
def scrape_jobs():
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    jobs = []
    
    for job_element in soup.find_all("div", class_="card-content"):
        job_title = job_element.find("h2", class_="title").text.strip()
        company_name = job_element.find("h3", class_="company").text.strip()
        location = job_element.find("p", class_="location").text.strip()
        description = job_element.find("div", class_="description").text.strip()
        application_link = job_element.find("a", string="Apply")["href"]
        jobs.append((job_title, company_name, location, description, application_link))
    return jobs

def store_jobs(jobs):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    for job in jobs:
        job_title, company_name, location, description, application_link = job
        cursor.execute(f"""
            INSERT INTO {TABLE_NAME} (job_title, company_name, location, description, application_link)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(job_title, company_name, location) DO UPDATE SET
                description = excluded.description,
                application_link = excluded.application_link;
        """, (job_title, company_name, location, description, application_link))
    
    conn.commit()
    conn.close()
def filter_jobs(location=None, company=None):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    query = f"SELECT * FROM {TABLE_NAME} WHERE 1=1"
    params = []
    
    if location:
        query += " AND location = ?"
        params.append(location)
    if company:
        query += " AND company_name = ?"
        params.append(company)
    
    cursor.execute(query, params)
    jobs = cursor.fetchall()
    conn.close()
    return jobs
def export_to_csv(filename, jobs):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title", "Company Name", "Location", "Description", "Application Link"])
        writer.writerows(jobs)

In [1]:
import requests
import json
from bs4 import BeautifulSoup

BASE_URL = "https://www.demoblaze.com/"
API_URL = "https://api.demoblaze.com/bycat"
HEADERS = {"Content-Type": "application/json"}
PAYLOAD = {"cat": "notebook"}  
def fetch_laptops():
    response = requests.post(API_URL, json=PAYLOAD, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("Items", [])
    return []
def parse_laptops(laptops):
    laptop_list = []
    for laptop in laptops:
        name = laptop.get("title", "Unknown")
        price = f"${laptop.get('price', 'N/A')}"
        description = laptop.get("desc", "No description available.")
        laptop_list.append({
            "name": name,
            "price": price,
            "description": description
        })
    return laptop_list
def save_to_json(data, filename="laptops.json"):
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    laptops = fetch_laptops()
    parsed_laptops = parse_laptops(laptops)
    save_to_json(parsed_laptops)
    print("Laptop data scraped and saved to laptops.json.")


Laptop data scraped and saved to laptops.json.
